In [1]:
import requests
from bs4 import BeautifulSoup
from scrapfly import ScrapflyClient, ScrapeConfig
import re
import json
import html

api_key = 'scp-live-01fc8534987042f4a2fe553c5cf6a2df'
scrapfly = ScrapflyClient(key=api_key, max_concurrency=2)
url = 'https://www.samys.com/p/4331/SmallRig-L-Series/NP-F550-USB-C-Rechargeable-Camera-Battery/261214.html'
result = scrapfly.scrape(ScrapeConfig(
    url=url,
    render_js=True,
    country="US",
    asp=True,
    retry=False,
    timeout=30000,
    rendering_wait=5000
))

html_content = result.scrape_result['content']


In [35]:
def extract_json_from_html_attributes(html_content):
   

    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Dictionary to store all extracted JSON data
    all_json_data = {}
    def parse_json_from_attribute(attr_value):
        # Unescape HTML entities
        unescaped_value = html.unescape(attr_value)
        
        # Try to parse the entire attribute value as JSON
        try:
            return json.loads(unescaped_value)
        except json.JSONDecodeError:
            # If that fails, try to find JSON-like structures
            #json_pattern = re.compile(r'\{[^{}]*\}')
            json_pattern = re.compile(r'\{(?:[^{}]|\\n|\\t|\\r)*\}', re.DOTALL)
            matches = json_pattern.findall(unescaped_value)
            
            # Try to parse each match
            for match in matches:
                try:
                    return json.loads(match)
                except json.JSONDecodeError:
                    # If parsing fails, try to expand the match to include nested structures
                    expanded_match = expand_json_match(unescaped_value, match)
                    try:
                        return json.loads(expanded_match)
                    except json.JSONDecodeError:
                        pass
        return None

    def expand_json_match(full_string, match):
        start_index = full_string.index(match)
        end_index = start_index + len(match)
        brace_count = 0
        
        # Expand to the left
        for i in range(start_index, -1, -1):
            if full_string[i] == '{':
                brace_count += 1
            elif full_string[i] == '}':
                brace_count -= 1
            if brace_count == 0:
                start_index = i
                break
        
        # Expand to the right
        brace_count = 0
        for i in range(end_index, len(full_string)):
            if full_string[i] == '{':
                brace_count += 1
            elif full_string[i] == '}':
                brace_count -= 1
            if brace_count == 0:
                end_index = i + 1
                break
        
        return full_string[start_index:end_index]

    # Find all elements
    all_elements = soup.find_all()

    for element in all_elements:
        for attr_name, attr_value in element.attrs.items():
            if isinstance(attr_value, str):  # Ensure the attribute value is a string
                json_data = parse_json_from_attribute(attr_value)
                if json_data:
                    if attr_name not in all_json_data:
                        all_json_data[attr_name] = []
                    all_json_data[attr_name].append(json_data)

    return all_json_data

   

In [36]:
json_data_list = extract_json_from_html_attributes(html_content)

file_path = r'C:\Users\SudheerRChinthala\circana\Srikanth\Data\SamysJSONData.json'
with open(file_path, 'w') as json_file:
    json.dump(json_data_list, json_file, indent=4)


In [37]:
json_data_list

{'content': [-1,
  2024,
  True,
  471693462881401,
  1,
  2,
  3,
  4,
  261214,
  44.99,
  44.99,
  44.99,
  144.0,
  339.0,
  49.95,
  58.95,
  279.0,
  7.75,
  599.0,
  149.99,
  295.0,
  165.0,
  199.0,
  3299.99],
 'gapi_processed': [True],
 'data-dom-ready': [True],
 'data-ind-ratio': [100],
 'data-resources-loaded': [True],
 'tabindex': [-1, 1, 1, 1, 5000, -1, -1, -1, -1, -1, -1, 5001, -1, -1, -1, -1],
 'aria-hidden': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 'aria-modal': [True],
 'aria-level': [2],
 'aria-haspopup': [True],
 'data-inddrag': [True],
 'data-menu-initialized': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True],
 'data-ajax': [{'url': '/modules/menus/top_ancillary_menu.jsp',
   'data': {'menuCategoryID': '1410',
    'subMenuBullets': False,
    'topWedge': True}},
  {'url

In [4]:
file_path = r'C:\Users\SudheerRChinthala\circana\Srikanth\Data\SamysJSONData.json'
with open(file_path, 'w') as json_file:
    for json_data in json_data_list:
        json.dump(json_data, json_file, indent=4)

In [5]:
json_data_list

[{'f': 'def9300743ceeee4e1963b140bc7f384',
  's': 'BRAINTREE_SIGNIN',
  'b': 'https://b.stats.paypal.com/counter.cgi?i=127.0.0.1&p=def9300743ceeee4e1963b140bc7f384&t=1727505783.603&a=14'},
 {}]